# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns

from joblib import load, dump
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sqlalchemy import create_engine

In [25]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/oamadeus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/oamadeus/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/oamadeus/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [26]:
# load data from database
database_filepath = "../disaster_response/data/DisasterResponse.db"
engine = create_engine(f"sqlite:///{database_filepath}")

num_of_feature_cols = 4

df = pd.read_sql_table("DisasterResponse", engine)

X = df["message"]
Y = df[df.columns[num_of_feature_cols:]]

In [28]:
classes_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [32]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(text)
    clean_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
pipeline = Pipeline([
    ("c_vect", CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2), max_df=0.95)),
    ("tfidf", TfidfTransformer(use_idf=True, smooth_idf=True)),
    ("clf", MultiOutputClassifier(RandomForestClassifier(verbose=1, n_jobs=6))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 

pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [55]:
def extract_macro_avg(report):
    for item in report.split("\n"):
        if "macro avg" in item:
            return float(item.strip().split()[4])

In [58]:
def show_scores(predicted_values):
    macro_avg_list = []
    
    for i in range(1, len(classes_names)):
        report = classification_report(Y_test.iloc[:, i].values, Y_pred[:, i], zero_division=1)
        macro_avg_list.append(extract_macro_avg(report))
        print("Category:", classes_names[i], "\n", report)
        
    overall_avg_score = sum(macro_avg_list) / len(macro_avg_list)
    print(f"Overral average score: {overall_avg_score:.3}")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 

Y_pred = model.predict(X_test)
show_scores(Y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    "c_vect__ngram_range": ((1, 1), (1, 2)),
    "c_vect__max_df": (0.75, 0.90, 1.0),
    "c_vect__max_features": (5000, 10000),
    "tfidf__use_idf": (True, False),
    "tfidf__smooth_idf": (True, False),
    "tfidf__sublinear_tf": (True, False),
    "clf__estimator__n_estimators": [300],
}

cv = GridSearchCV(pipeline, parameters)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 

### BE CAREFUL! This may take a very very long time to fit ###
#cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [86]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 
model = load("../disaster_response/models/model.pkl")

Y_pred = model.predict(X_test)
show_scores(Y_pred)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.3s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      |

Category: request 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      4349
           1       0.99      0.88      0.93       895

    accuracy                           0.98      5244
   macro avg       0.98      0.94      0.96      5244
weighted avg       0.98      0.98      0.98      5244

Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       1.00      0.46      0.63        26

    accuracy                           1.00      5244
   macro avg       1.00      0.73      0.82      5244
weighted avg       1.00      1.00      1.00      5244

Category: aid_related 
               precision    recall  f1-score   support

           0       0.95      0.98      0.96      3113
           1       0.97      0.92      0.94      2131

    accuracy                           0.95      5244
   macro avg       0.96      0.95      0.95      5244
weighted avg

In [66]:
def feature_importances(coef, features_names, n_top=-1):
    """Plots horizontal bar plot with n_top most significant features used in prediction"""
    coef, features_names = zip(*sorted(list(zip(coef, features_names))))

    # Show all features
    if n_top == -1:
        n_top = len(features_names)
    
    plt.rcParams["figure.figsize"] = (10,7)
    plt.barh(range(n_top), coef[::-1][0:n_top], align='center')
    plt.yticks(range(n_top), features_names[::-1][0:n_top])
    plt.show()

In [79]:
feature_importances(abs(clf.coef_[0]), cv.get_feature_names(), n_top=20)

AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'

In [80]:
clf = model.estimator["clf"].estimator
clf.c

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [77]:
cv = model.estimator["c_vect"]

/home/oamadeus/repositories/ds_env/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.95, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x7f61c1452d90>,
                vocabulary=None)

['!',
 '! !',
 '! #',
 '! $',
 '! &',
 "! '",
 "! ''",
 '! (',
 '! )',
 '! ,',
 '! -blankets',
 '! .',
 '! ..',
 '! .man',
 '! /',
 '! 0_0',
 '! 5',
 '! 7.0',
 '! 77h',
 '! 911',
 '! :',
 '! ?',
 '! @',
 '! \\nthanks',
 '! _',
 '! __',
 '! ___w/',
 '! ``',
 '! a',
 '! after',
 '! all',
 '! aluminium',
 '! and',
 '! another',
 '! any',
 '! arsenic',
 '! aushduiaiuas',
 '! baby',
 '! back',
 '! be',
 '! better',
 '! bisou',
 '! bless',
 '! boron',
 '! bring',
 '! bryllum',
 '! business',
 '! but',
 '! buy',
 '! ca',
 '! cabon',
 '! calcium',
 '! can',
 '! checking',
 '! chile',
 '! chillan',
 '! clean',
 '! cleaning',
 '! come',
 '! complete',
 '! concepcion',
 '! coned',
 '! cop',
 '! could',
 '! couldnt',
 '! counting',
 '! courage',
 '! cousin',
 '! damn',
 '! den',
 '! digicel',
 '! do',
 '! domagecles',
 '! drink',
 '! effecte',
 '! eph',
 '! every',
 '! first',
 '! fixxxxxx',
 '! flew',
 '! fontamara',
 '! food',
 '! forgive',
 '! friend',
 '! fuck',
 '! fuerza',
 '! g',
 '! galliu

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class PosCounter(BaseEstimator, TransformerMixin):
    
    def pos_tagger(self, text):
        tokenized = tokenize(text)

        tagged = nltk.pos_tag(tokenized)

        part_of_speech_list = []

        for word_tag_pair in tagged:
            tag = word_tag_pair[1]
            if tag[0] == "V":
                part_of_speech_list.append("verb")
            elif tag[0] == "N":
                part_of_speech_list.append("noun")
            elif tag[0] == "J":
                part_of_speech_list.append("adjective")

        return part_of_speech_list
    
    def count_pos(self, l, pos):
        return l.count(pos)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = pd.DataFrame(X.apply(self.pos_tagger))
        
        for pos in ("verb", "noun", "adjective"):
            X[pos] = X["message"].apply(lambda l: self.count_pos(l, pos))

        del X['message']
        
        return X

In [ ]:
pc = PosCounter()
pc.transform(X)

In [ ]:
pipeline = Pipeline([
    ("features", FeatureUnion([
        ("nlp_pipeline", Pipeline([
            ("c_vect", CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2), max_df=0.95)),
            ("tfidf", TfidfTransformer(use_idf=True, smooth_idf=True)),
        ])),

        ("pos_counter", PosCounter()),
    ])),
    ("clf", MultiOutputClassifier(LinearSVC(verbose=2, max_iter=100000))),
])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 

pipeline.fit(X_train, Y_train)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42) 

Y_pred = pipeline.predict(X_test)
show_scores(Y_pred)

### 9. Export your model as a pickle file

In [ ]:
dump(pipeline, "../disaster_response/models/new_model.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.